# Versiones Públicas de Sentencias - Divorcios

Instale en su entorno virtual los paquetes `pip install python-dotenv requests pandas ipykernel`

Debe tener una archivo `.env` donde guarde de forma segura su USERNAME y PASSWORD.

In [1]:
from dotenv import load_dotenv
from IPython.display import display
import os
import pandas as pd
import requests

In [2]:
# Hola a Plataforma Web API OAuth2
HOST = "http://localhost:8002"
respuesta = requests.get(HOST)
print(respuesta.status_code)
print(respuesta.text)

KeyboardInterrupt: 

In [ ]:
# Cargar username/password
load_dotenv()
parametros = {
    "username": os.getenv("USERNAME"),
    "password": os.getenv("PASSWORD"),
}
print(parametros["username"])

In [ ]:
# Solicitar token
cabeceras_para_autentificar = {'content-type': "application/x-www-form-urlencoded"}
respuesta = requests.post(f"{HOST}/token", data=parametros, headers=cabeceras_para_autentificar)
print(respuesta.status_code)

In [ ]:
# Definir cabeceras_para_api
cabeceras_para_api = None
if respuesta.status_code == 200:
    token = respuesta.json()['access_token']
    cabeceras_para_api = {'Authorization': 'Bearer ' + token}
print(cabeceras_para_api)

In [ ]:
# Función consultar
def consultar(ruta: str, parametros: dict = None):
    """Consultar la API entrega dos valores: listado/registro, total"""
    try:
        respuesta = requests.get(f"{HOST}/v1/{ruta}", headers=cabeceras_para_api, params=parametros, timeout=12)
    except requests.exceptions.RequestException as error:
        raise Exception("Parece que hay un error en la consulta: " + str(error))
    #print(respuesta.status_code)
    if respuesta.status_code != 200:
        raise requests.HTTPError("Parece que ya caducó el token")
    datos = respuesta.json()
    if "items" in datos:
        return pd.json_normalize(datos["items"]), datos["total"]
    return pd.json_normalize(datos), 1

In [ ]:
# Soportes categorías
soportes_categorias, total = consultar("soportes_categorias")
soportes_categorias.head(8)

In [ ]:
# Rango de fechas a consultar
FECHA_DESDE = "2022-02-14"
FECHA_HASTA = "2022-02-15"
# Consultar tickets
parametros = {"fecha_desde": FECHA_DESDE, "fecha_hasta": FECHA_HASTA, "limit": 50}
soportes_tickets, total = consultar("soportes_tickets", parametros)
print("Total", total)
soportes_tickets.head(8)
